In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np
import re
import matplotlib.pyplot as plt

import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

from time import sleep

from skimage import io

from concurrent.futures import ThreadPoolExecutor

import json
import pickle

import os

from glob import glob
from datetime import datetime, timedelta

In [2]:
def get_soup_simple(url, strain_name=None, strain_attrs=None):
    response = requests.get(url)#, headers=headers, proxies=proxies)
    if strain_name is None and strain_attrs is None:
        soup = BeautifulSoup(response.text, 'lxml')
    elif strain_attrs is None:
        strainer = SoupStrainer(strain_name)
        soup = BeautifulSoup(response.text, 'lxml', parse_only=strainer)
    else:
        strainer = SoupStrainer(strain_name, strain_attrs)
        soup = BeautifulSoup(response.text, 'lxml', parse_only=strainer)
    return soup

In [3]:
base_url = 'https://arachno.piwigo.com/'
families_url = base_url + 'index?/category/familles'

main_page = get_soup_simple(families_url, 'li', {'class':'selected liOpen'})

In [35]:
info = []

for f in tqdm(main_page.li('li')):
    genus_info = []
    try:
        if f['class'][0] == 'liClosed':
            family = f.a.getText()
            for g in f('li'):
                genus_arr = g.a.getText().split(' ')
                genus = genus_arr[0]
                
                species = genus
                if len(genus_arr) > 1:
                    species = genus + ' ' + ' '.join(genus_arr[1:])
                
                species_link = base_url + f'{g.a["href"]}' 
                
                info.append([family, genus, species, species_link])
    except:
        None
        
info = np.array(info)

df = pd.DataFrame(info, columns=['family', 'genus', 'species', 'link'])
df.head()

,family,genus,species,link
0,Agelenidae,Agelena,Agelena agelenoides,https://arachno.piwigo.com/index?/category/176...
1,Agelenidae,Agelena,Agelena labyrinthica,https://arachno.piwigo.com/index?/category/67-...
2,Agelenidae,Allagelena,Allagelena gracilens,https://arachno.piwigo.com/index?/category/68-...
3,Agelenidae,Aterigena,Aterigena ligurica,https://arachno.piwigo.com/index?/category/143...
4,Agelenidae,Aterigena,Aterigena soriculata,https://arachno.piwigo.com/index?/category/159...


In [36]:
df.shape

(1465, 4)

In [37]:
df.to_json('data/spiderbase_0.json')

In [38]:
df = pd.read_json('data/spiderbase_0.json')

In [39]:
df.shape

(1465, 4)

In [40]:
def group_images(img_content, species):
    groups = {'general': [], 'epigyne': [], 'palpe': [], 'vulve': [], 'other': []}
    for img in img_content:
        name = img['name'].lower()
        if 'epigyne' in name:
            groups['epigyne'].append(img)
        elif 'palpe' in name:
            groups['palpe'].append(img)
        elif 'vulve' in name:
            groups['vulve'].append(img)
        elif name == species.lower():
            groups['general'].append(img)
        else:
            groups['other'].append(img)
    return groups

# def get_image_metainfo(args):
#     url, species_name = args
#     img_content = []
#     sleep(1)
#     try:
#         img_data = get_soup_simple(url, 'ul', {'class':'thumbnails', 'id':"thumbnails"})('span', 'wrap2')
#         for dt in img_data:
#             sleep(1)
#             img_site_link = base_url+dt.a['href']
#             name = dt.img.attrs['alt']
#             credits = dt.img.attrs['title']
#             main_image = get_soup_simple(img_site_link, 'img', {'id': 'theMainImage'}).img
#             link = base_url+main_image.attrs['src']
#             image = io.imread(link)
#             img_content.append({'name': name, 'image': image , 'credits': credits})

#         groups = group_images(img_content, species_name)
#         #pickle.dump(groups, open(f'data/image_data/{species_name.replace(" ","-").lower()}.pkl','wb'))
#         print('fix this')
#         return True, args, ''
#     except Exception as e:
#         return False, args, e
    
def get_image_metainfo_quick(args):
    url, species_name = args
    img_content = []
    sleep(1)
    try:
        img_data = get_soup_simple(url, 'ul', {'class':'thumbnails', 'id':"thumbnails"})('span', 'wrap2')
        for dt in img_data:
            sleep(0.1)
            name = dt.img.attrs['alt']
            credits = dt.img.attrs['title']
            link = base_url+dt.img['src']
            image = io.imread(link)
            img_content.append({'name': name, 'image': image , 'credits': credits})

        groups = group_images(img_content, species_name)
        with open(f'data/image_data_0/{species_name.replace(" ","-").lower()}.pkl','wb') as out_file:
            pickle.dump(groups, out_file)
        return True, args, ''
    except Exception as e:
        return False, args, e

## Remove data which is deprecated

In [42]:
existing_data = glob('data/image_data_0/*.pkl')
updated_data = [f'data/image_data_0/{species_name.replace(" ","-").lower()}.pkl' for species_name in df.species]

for data in existing_data:
    if data not in updated_data:
        print(f'Deleting {data}')
        os.system(f'rm -rf {data}')

## Download any new data

In [44]:
arg_list= df[['link', 'species']].values.astype(str)

results = []
for arg in tqdm(arg_list):
    species_name = arg[1]
    fname = f'data/image_data_0/{species_name.replace(" ","-").lower()}.pkl'
    if not os.path.isfile(fname):
        print(f'Downloading: {species_name}')
        results.append(get_image_metainfo_quick(arg))

Downloading: Erigone
Downloading: Meioneta
Downloading: Oedothorax
Downloading: Porrhomma
Downloading: Tenuiphantes
Downloading: Walckenaeria
Downloading: x-Phantes



## Re-download updated images

In [53]:
latest_update = '19-03-2020'
latest_update = datetime.strptime(latest_update, '%d-%m-%Y').date()

today = datetime.today().date()

def get_update_page(date):
    return f'https://arachno.piwigo.com/index?/category/familles/posted-monthly-list-{date.year}-{date.month}-{date.day}'

In [57]:
dates = []
species = []

all_updates = False
i = 0

while not all_updates:
    if latest_update == today:
        break
        
    if i == 0:
        date = today
    
    sleep(1)
    data = get_soup_simple(get_update_page(date))
    print(f'Number of parsed pages: {i+1}')
    
    try:
        species.extend([x['alt'].split('-')[0].rstrip() for x in data('ul', 'thumbnails')[0]('img')])
    except:
        None
    link = base_url+data('div', 'calendarBar')[0].div.a['href']
    prev_date = datetime.strptime('-'.join(link.split('-')[-3:]), '%Y-%m-%d').date()

    if prev_date < latest_update:
        all_updates = True
    else:
        date = prev_date
        i += 1
    
species = [x for x in species if len(x.split(' ')) <= 3]
species = np.unique(species)

Number of parsed pages: 1
Number of parsed pages: 2


In [60]:
arg_list_update = df[df.species.isin(species)][['link', 'species']].values.astype(str)

results = []
for arg in tqdm(arg_list_update):
    print(f'Updating: {arg[1]}')
    results.append(get_image_metainfo_quick(arg))

Downloading: Zelotes egregioides
Downloading: Peponocranium ludicrum
Downloading: Argyrodes argyrodes
Downloading: Coleosoma floridanum
Downloading: Cryptachaea blattea
Downloading: Kochiura aulica

